# PV-IN KCNC2 Kv3.2 A new genetic cause of childhood epilepsy


### imports and setup

In [1]:
%load_ext autoreload
%autoreload 2
import logging

from pathlib import Path
from functools import lru_cache
from itertools import product
from neuron import h
from neuron import gui
from tqdm import tqdm
from typing import Iterable, Union
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.cells.pv_nrn import get_pv, get_pv_params, reset_biophys

from src.constants import *
from src.settings import *
from src.data import get_cached_df, get_file_path, set_cache_root, get_cache_root, wide_to_long, concise_df
from src.measure import get_max_propagation, get_ap_times, calculate_failures
from src.run import get_trace, set_relative_nav11bar, set_nrn_prop
from src.utils import get_key, format_nav_loc, perc_decrease, str_to_tuple, nearest_idx, nearest_value, get_last_sec
from src.vis import plot_voltage_trace, set_default_style, save_fig, get_pulse_times, get_pulse_xy
from src.nrn_helpers import init_nrn

logging.basicConfig(level=logging.INFO)

set_cache_root("D:\\.cache")

init_nrn(celsius=34, v_init=-80) # as in BBP optimisation

h.check_simulator()

set_default_style()

INFO:hoc_helpers:celsius=34.0 and v_init=-80.0


## Set up

In [4]:
pv = get_pv()
pv

pv[0]